# Compute $a^{c}_\mu$ and $a^{b}_\mu$

In [1]:
from adlerpy.adler_sm import adler_charm_pert
from adlerpy.adler_sm import adler_bottom_pert
from scipy.optimize import fsolve

from adlerpy.adler_routines import Particle
from scipy.integrate import quad
from matplotlib import pyplot as plt 
import numpy as np 
import pandas as pd


#Create the particles
aZ=0.1185;
Mz=91.1876;
alpha0=1/137.035999180;


#Here you define the SM particles 

up=Particle("up",x=[2.16*0.001,2,3],mudec=0.001,mpole=None,mpole_on=False)
down=Particle("down",x=[4.67*0.001,2,3],mudec=0.001,mpole=None,mpole_on=False)
strange=Particle("strange",x=[0.09203,2,3],mudec=0.001,mpole=None,mpole_on=False)
charm=Particle("charm",x=[1.273,1.273,4],mudec=2*1.273,mpole=1.67,mpole_on=False)
bottom=Particle("bottom",x=[4.18,4.18,5],mudec=2*4.18,mpole=4.25,mpole_on=False)
top=Particle("top",x=[164,164,6],mudec=164,mpole=164,mpole_on=False)
particle_list=[up,charm,down,strange,bottom,top] 


## Define the functions to integrate 

In [2]:
def integrand_ac(Q2,aZ,mc,mpole_on=False,nloops=5,QED=False,GG=0,cut_high_as3=20,cut_low_as3=0.3):
    ml=0.1057
    u=np.sqrt(1 + (4*ml**2)/Q2);
    kl=(u-1)**2/(u+1)**2/2
    Q=np.sqrt(Q2);
    
    charm_intermidiate=Particle("charm",x=[mc,mc,4],mudec=2*mc,mpole=1.67,mpole_on=False)
    particle_list_intermediate=[up,charm_intermidiate,down,strange,bottom,top] 

    ad=(adler_charm_pert(aZ,Mz,Q=Q,
                         particles=particle_list_intermediate,
                         mpole_on=mpole_on,
                         mu=None,
                         nloops=nloops,
                         cut_high_as3=cut_high_as3,
                         cut_low_as3=cut_low_as3,
                         QED=QED,
                         GG=GG))/Q2
    return (ad*kl)/3;

def integrand_ab(Q2,aZ,mb,mpole_on=False,nloops=5,QED=False,GG=0,cut_high_as3=20,cut_low_as3=0.3):
    ml=0.1057
    u=np.sqrt(1 + (4*ml**2)/Q2);
    kl=(u-1)**2/(u+1)**2/2
    Q=np.sqrt(Q2);
    bottom_intermediate=Particle("bottom",x=[mb,mb,5],mudec=2*mb,mpole=4.25,mpole_on=False)
    particle_list_intermidiate=[up,charm,down,strange,bottom_intermediate,top] 

    ad=(adler_bottom_pert(aZ,Mz,Q=Q,
                         particles=particle_list_intermidiate,
                         mpole_on=mpole_on,
                         mu=None,
                         nloops=nloops,
                         cut_high_as3=cut_high_as3,
                         cut_low_as3=cut_low_as3,
                         QED=QED,
                         GG=GG))/Q2
    #ad=(1/3)**4*(3/4*alpha0/np.pi*adler_hq_full_one_loop_MS(Q,4.18,4.18))/Q2
    return (ad*kl)/3

## Compute $a^{(c)}_\mu$

In [3]:
QED=True;
mpole_on=False
GG=0;
nloops=3;
mc=1.273
amuc=(alpha0/np.pi)**2*quad(integrand_ac, 0.0, np.infty,args=(aZ,mc,mpole_on,nloops,QED,GG))[0]*10**10
amuc_low=(alpha0/np.pi)**2*quad(integrand_ac, 0.0, np.infty,args=(aZ,mc,mpole_on,nloops-1,QED,GG))[0]*10**10

#-----------# the change in mc in the following line comes from the  correlation seen in  https://arxiv.org/pdf/1610.08531.pdf
amuc_aplus=(alpha0/np.pi)**2*quad(integrand_ac, 0.0, np.infty,args=(aZ+0.0016,mc+2.616*0.0016,mpole_on,nloops,QED,GG))[0]*10**10 


amuc_mcplus=(alpha0/np.pi)**2*quad(integrand_ac, 0.0, np.infty,args=(aZ,mc+0.008,mpole_on,nloops,QED,GG))[0]*10**10

In [4]:
text=("a^{(c)}_\\mu\\times10^{10}= "+ str(round(amuc,2))+
      "\pm"+
      str(round(np.abs(amuc_aplus-amuc),2))+
      "_{\\alpha_s}\pm"+
      str(round(np.abs(amuc_mcplus-amuc),2))+
      "_{m_c} \pm"+
      str(round(np.abs(amuc_low-amuc),2))+"_{\mathrm{tr}}"
     )
from IPython.display import display, Math
display(Math(text))


<IPython.core.display.Math object>

## Compute $a^{(b)}_\mu$

In [5]:
QED=True;
mpole_on=False
GG=0;
nloops=3;
mb=4.18
amub=(alpha0/np.pi)**2*quad(integrand_ab, 0, np.infty,args=(aZ,mb,mpole_on,nloops,QED,GG))[0]*10**10
amub_low=(alpha0/np.pi)**2*quad(integrand_ab, 0, np.infty,args=(aZ,mb,mpole_on,nloops-1,QED,GG))[0]*10**10
amub_aplus=(alpha0/np.pi)**2*quad(integrand_ab, 0, np.infty,args=(aZ+0.0016,mb,mpole_on,nloops,QED,GG))[0]*10**10 
amub_mbplus=(alpha0/np.pi)**2*quad(integrand_ab, 0, np.infty,args=(aZ,mb+0.008,mpole_on,nloops,QED,GG))[0]*10**10

In [6]:
text=("a^{(b)}_\\mu\\times10^{10}= "+ str(round(amub,3))+
      "\pm"+
      str(round(np.abs(amub_aplus-amub),3))+
      "_{\\alpha_s}\pm"+
      str(round(np.abs(amub_mbplus-amub),3))+
      "_{m_b} \pm"+
      str(round(np.abs(amub_low-amub),3))+"_{\mathrm{tr}}"
     )
from IPython.display import display, Math
display(Math(text))


<IPython.core.display.Math object>